In [1]:
import yt
import os
import cProfile
import pandas as pd 

ddir = "/home/chavlin/src/temp/"

/home/chavlin/src/yt/yt/utilities/logger.py:4: VisibleDeprecationWarning: The configuration file /home/chavlin/.config/yt/ytrc is deprecated in favor of /home/chavlin/.config/yt/yt.toml. Currently, both are present. Please manually remove the deprecated one to silence this warning.
Deprecated since v4.0.0 . This feature will be removed in v4.1.0
  from yt.config import ytcfg


function to convert cProfile output to pandas dataframe:

In [2]:
def get_df(pr):
    attrs = ['code','callcount','reccallcount','totaltime','inlinetime','calls']
    rows = []
    for st in pr.getstats():
        rows.append([getattr(st,attr) for attr in attrs])
        
    return pd.DataFrame(rows,columns=attrs)

right now I have some environment variable flags to switch between the different particle reader methods. This is a temporary hacky solution for switching between the different methods, which are:


* `'original'` : the normal reader, does a count, pre-allocate then fill 
* `'dask_array'`: does a count (with dask), builds dask arrays `from_delayed()`, converts to in-memory 
* `'dask_dataframe'`: no count, builds dask dataframe (each column is a field), converts to `dask_array`, converts to in-memory array
* `'dask_dataframe_valsonly'`: no count, builds dask dataframe, converts to in memory df, extracts values to np arrays (no intermediate conversion to `dask_dataframe`)

So the following code loads a dataset then profiles just the selection from `ds.region` 

In [3]:
methnames = ['original','dask_array','dask_dataframe','dask_dataframe_valsonly']
ds = yt.load_sample("snapshot_033") # load it
ndx = ds.index # make sure the index is initialized (otherwise this happens when selecting)

for meth in methnames:
    os.environ['yt_dask_flag'] = meth
    # creating a new region seems sufficient for clearing any cache...
    reg = ds.region(ds.domain_center,ds.domain_right_edge*0.25, ds.domain_right_edge*0.75)
    with cProfile.Profile() as pr:
        den = reg[('PartType4', 'Density')]
    df = get_df(pr)
    df.to_csv(os.path.join(ddir,f"prof_stats_{meth}.csv"),index=False,sep="|")

yt : [INFO     ] 2021-02-10 11:05:07,294 Files located at /home/chavlin/hdd/data/yt_data/yt_sample_sets/snapshot_033.tar.gz.untar/snapshot_033/snap_033.
yt : [INFO     ] 2021-02-10 11:05:07,295 Default to loading snap_033.0.hdf5 for snapshot_033 dataset
yt : [INFO     ] 2021-02-10 11:05:07,356 Parameters: current_time              = 4.343952725460923e+17 s
yt : [INFO     ] 2021-02-10 11:05:07,357 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2021-02-10 11:05:07,357 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2021-02-10 11:05:07,357 Parameters: domain_right_edge         = [25. 25. 25.]
yt : [INFO     ] 2021-02-10 11:05:07,358 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2021-02-10 11:05:07,358 Parameters: current_redshift          = -4.811891664902035e-05
yt : [INFO     ] 2021-02-10 11:05:07,359 Parameters: omega_lambda              = 0.762
yt : [INFO     ] 2021-02-10 11:05:07,359 Parameters: omega_matter              = 0.238
yt :

    original count
original method
dask array
dask_df
dask_df
    direct vals


So now we can load back in the dataframes for each method:

In [4]:
df_dict = {meth:pd.read_csv(os.path.join(ddir,f"prof_stats_{meth}.csv"),sep='|') for meth in methnames}

In [5]:
df_dict['original'].sort_values(by='inlinetime',ascending=False).head()

,code,callcount,reccallcount,totaltime,inlinetime,calls
44,<method 'identify_file_masks' of 'yt.geometry....,1,0,5.066298,5.066096,[_lsprof.profiler_subentry(code=<code object w...
32,<code object _read_particle_fields at 0x7f9ebf...,9,0,0.007993,0.001622,[_lsprof.profiler_subentry(code='<built-in met...
69,"<code object __getitem__ at 0x7f9ece8d03a0, fi...",24,0,0.004168,0.001308,[_lsprof.profiler_subentry(code='<built-in met...
87,"<code object make_fid at 0x7f9ecf3bd7c0, file ...",20,0,0.001305,0.001260,[_lsprof.profiler_subentry(code=<code object p...
94,"<code object __getitem__ at 0x7f9ecf3c1c90, fi...",32,0,0.002357,0.001206,[_lsprof.profiler_subentry(code=<code object p...


In [6]:
df_dict['dask_array'].sort_values(by='inlinetime',ascending=False).head()

,code,callcount,reccallcount,totaltime,inlinetime,calls
113,<method 'identify_file_masks' of 'yt.geometry....,1,0,5.103516,5.103111,[_lsprof.profiler_subentry(code=<code object w...
64,<method 'acquire' of '_thread.lock' objects>,141,0,0.025943,0.025943,NaN
47,<built-in method posix.urandom>,40,0,0.000543,0.000543,NaN
463,"<code object where at 0x7f9f00114b30, file ""<_...",11,0,0.000405,0.000381,[_lsprof.profiler_subentry(code=<code object w...
341,"<code object keys_in_tasks at 0x7f9eca20cb30, ...",127,0,0.000264,0.000242,[_lsprof.profiler_subentry(code='<built-in met...


In [7]:
df_dict['dask_dataframe'].sort_values(by='inlinetime',ascending=False).head()

,code,callcount,reccallcount,totaltime,inlinetime,calls
79,<method 'identify_file_masks' of 'yt.geometry....,1,0,5.095822,5.095455,[_lsprof.profiler_subentry(code=<code object w...
51,<method 'acquire' of '_thread.lock' objects>,91,0,0.037567,0.037567,NaN
575,"<code object keys_in_tasks at 0x7f9eca20cb30, ...",91,0,0.000501,0.000386,[_lsprof.profiler_subentry(code='<built-in met...
706,"<code object where at 0x7f9f00114b30, file ""<_...",11,0,0.000367,0.000318,[_lsprof.profiler_subentry(code=<code object w...
626,"<code object get_data at 0x7f9ed28260e0, file ...",1,0,5.142374,0.000238,[_lsprof.profiler_subentry(code='<built-in met...


In [8]:
df_dict['dask_dataframe_valsonly'].sort_values(by='inlinetime',ascending=False).head()

,code,callcount,reccallcount,totaltime,inlinetime,calls
74,<method 'identify_file_masks' of 'yt.geometry....,1,0,5.057266,5.056845,[_lsprof.profiler_subentry(code=<code object w...
47,<method 'acquire' of '_thread.lock' objects>,77,0,0.036149,0.036149,NaN
654,"<code object where at 0x7f9f00114b30, file ""<_...",11,0,0.000420,0.000384,[_lsprof.profiler_subentry(code=<code object w...
525,"<code object keys_in_tasks at 0x7f9eca20cb30, ...",105,0,0.000498,0.000377,"[_lsprof.profiler_subentry(code=""<method 'exte..."
38,<built-in method posix.urandom>,22,0,0.000294,0.000294,NaN


to identify the calls that are taking more time compared to the original, we can merge the dataframes and calculate some differences. Using a left join of the original on the new, we can identify the additional calls coming from dask. The resulting dataframes will have `_n` and `_o` suffixes for the new and original method values, respectively, and `d_` prefix for the differences:

In [9]:
diff_cols =['callcount','inlinetime','totaltime']
df_o = df_dict['original']
for meth in ['dask_array','dask_dataframe','dask_dataframe_valsonly']:
    # merge w original 
    df_n = df_dict[meth]
    df_new = df_n.merge(df_o,how='left',on='code',suffixes=('_n','_o')).fillna(0.)
    
    # calculate differences for the following columns
    diff_cols =['callcount','inlinetime','totaltime']
    for col in diff_cols:
        df_new['d_'+col]=df_new[col+'_n'] - df_new[col+'_o']
    df_new.to_csv(os.path.join(ddir,f"merged_stats_{meth}.csv"),sep='|',index=False)

and let's load those back in, sort by the difference in inline time:

In [10]:
diff_ds = {}
for meth in ['dask_array','dask_dataframe','dask_dataframe_valsonly']:
    diff_ds[meth] = pd.read_csv(os.path.join(ddir,f"merged_stats_{meth}.csv"),sep='|').sort_values(by='d_inlinetime',ascending=False)
    

In [11]:
diff_ds['dask_array'][['code','d_callcount','d_totaltime','d_inlinetime','callcount_o']].head()

,code,d_callcount,d_totaltime,d_inlinetime,callcount_o
113,<method 'identify_file_masks' of 'yt.geometry....,0.0,0.037218,0.037015,1.0
64,<method 'acquire' of '_thread.lock' objects>,136.0,0.025938,0.025938,5.0
47,<built-in method posix.urandom>,40.0,0.000543,0.000543,0.0
341,"<code object keys_in_tasks at 0x7f9eca20cb30, ...",127.0,0.000264,0.000242,0.0
463,"<code object where at 0x7f9f00114b30, file ""<_...",0.0,0.000203,0.000202,11.0


In [12]:
diff_ds['dask_array']['d_inlinetime'].sum()

0.0708528320000005

In [13]:
diff_ds['dask_dataframe'][['code','d_callcount','d_totaltime','d_inlinetime','callcount_o']].head()

,code,d_callcount,d_totaltime,d_inlinetime,callcount_o
51,<method 'acquire' of '_thread.lock' objects>,86.0,0.037562,0.037562,5.0
79,<method 'identify_file_masks' of 'yt.geometry....,0.0,0.029524,0.029360,1.0
575,"<code object keys_in_tasks at 0x7f9eca20cb30, ...",91.0,0.000501,0.000386,0.0
41,<built-in method posix.urandom>,22.0,0.000216,0.000216,0.0
515,<code object unpack_collections at 0x7f9eca1b2...,140.0,0.000732,0.000216,0.0


In [14]:
diff_ds['dask_dataframe']['d_inlinetime'].sum()

0.07355959900000031

In [15]:
diff_ds['dask_dataframe_valsonly'][['code','d_callcount','d_totaltime','d_inlinetime','callcount_o']].head()

,code,d_callcount,d_totaltime,d_inlinetime,callcount_o
47,<method 'acquire' of '_thread.lock' objects>,72.0,0.036144,0.036144,5.0
525,"<code object keys_in_tasks at 0x7f9eca20cb30, ...",105.0,0.000498,0.000377,0.0
38,<built-in method posix.urandom>,22.0,0.000294,0.000294,0.0
469,<code object unpack_collections at 0x7f9eca1b2...,140.0,0.000719,0.000217,0.0
654,"<code object where at 0x7f9f00114b30, file ""<_...",0.0,0.000218,0.000206,11.0


In [16]:
diff_ds['dask_dataframe_valsonly']['d_inlinetime'].sum()

0.03293069299999952

These values differ between runs somewhat... the `identify_file_masks` time is pretty variable sometimes it's slower/faster for the new methods. The `_thread.lock` calls are pretty consistent though. I could do all this in a loop and average each method's dfs over each run... 